# 20210119
- review image 데이터를 정답지(Y, N)으로 분류하여 별도 디렉토리에 저장

In [1]:
# 주피터노트북 화면 넓게 보기 위한 용도
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import boto3
import s3fs

import pymysql
from PIL import Image

from dotenv import load_dotenv
import os
load_dotenv() # .env 파일에서 정보를 갖고 오기 위함

In [18]:
!pip install python-dotenv
# https://medium.com/@namseok.yoo/django-%EC%97%90%EC%84%9C-python-dotenv-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0-6d9b58a939ec

In [5]:
conn = pymysql.connect(database=os.getenv('database'), user=os.getenv('user'), password=os.getenv('password'), host= 'localhost', port=3306)
qry = '''
    SELECT a.goods_no, a.est_no, b.image, a.point_yn
    FROM 
    (
        SELECT goods_no, no as est_no, point_yn
        FROM goods_estimate
        WHERE date_format(regi_date, '%Y%m%d') = '20201201'
        AND type = 'style'
    ) a
    JOIN
    (
        SELECT est_no, image
        FROM goods_estimate_image
        WHERE date_format(rt, '%Y%m%d') = '20201201'
        GROUP BY est_no, image HAVING count(img_no) = 1
    ) b
    ON a.est_no = b.est_no
'''
df = pd.read_sql_query(qry, conn)

In [12]:
df['point_yn'].describe()

count     6822
unique       2
top          Y
freq      5945
Name: point_yn, dtype: object

In [22]:
df.head()

,goods_no,est_no,image,point_yn
0,644025,3105258,/data/estimate/644025_0/gallery_5fc52d35e78a6.jpg,N
1,1003751,5882494,/data/estimate/1003751_0/gallery_5fc52c8300d03...,N
2,542281,6067380,/data/estimate/542281_0/gallery_5fc5fea7d5f7d.jpg,Y
3,841463,6769001,/data/estimate/841463_0/gallery_5fc52c322bde6.jpg,N
4,1198380,7641604,/data/estimate/1198380_0/gallery_5fc5a39d378f0...,Y


In [ ]:
def get_imgage(goods_no, img_path):
    file = boto3.resource('s3').Object('musinsa', img_path[1:]).get()
    img = np.array(Image.open(file['Body']))
    return img